Here, we will be building a DANN.

In [61]:
#importing modules
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, Model
import time
import matplotlib.pyplot as plt
import numpy as np
import keras as K
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.layers import Dense


In [72]:

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [73]:
#looking into data and its columns
import pandas as pd

# Replace 'your_file_path.csv' with the actual path to your CSV file
file_path = 'datasets/source/kaggle_food.csv'

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to check the data
print(df.head())


   Unnamed: 0                                              Title  \
0           0  Miso-Butter Roast Chicken With Acorn Squash Pa...   
1           1                    Crispy Salt and Pepper Potatoes   
2           2                        Thanksgiving Mac and Cheese   
3           3                 Italian Sausage and Bread Stuffing   
4           4                                       Newton's Law   

                                         Ingredients  \
0  ['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...   
1  ['2 large egg whites', '1 pound new potatoes (...   
2  ['1 cup evaporated milk', '1 cup whole milk', ...   
3  ['1 (¾- to 1-pound) round Italian loaf, cut in...   
4  ['1 teaspoon dark brown sugar', '1 teaspoon ho...   

                                        Instructions  \
0  Pat chicken dry with paper towels, season all ...   
1  Preheat oven to 400°F and line a rimmed baking...   
2  Place a rack in middle of oven; preheat to 400...   
3  Preheat oven to 350°F with 

In [74]:
#here we will only be using 'Title' and 'Ingredients' for our purpose
#so we will be dropping the remaining columns
df = df.drop(columns=['Unnamed: 0', 'Ingredients', 'Image_Name',
       'Cleaned_Ingredients'])

In [75]:
#here we are creating a fake labels for temporary test
#1/3 part of the labels to 'appetizers'. another 1/3 part to 'dinner' and the last 1/3 part to 'desserts'

total_size = len(df)
category_size = total_size // 3

df.loc[:category_size - 1, 'Title'] = 'Appetizers'
df.loc[category_size:2*category_size - 1, 'Title'] = 'Dinner'
df.loc[2*category_size:total_size - 1, 'Title'] = 'Desserts'

In [76]:
#shuffling the dataframe
df = df.sample(frac=1, random_state=42)

# Assuming df['Instructions'] is your text data
df['Instructions'].fillna('', inplace=True)  # Replace NaN values with an empty string


In [77]:
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

#defining the parameters
num_classes = 1
embedding_dim = 100

# Assuming df['Instructions'] is your text data, we  tokenize the input dataset into tokens for the CNN model
#Tokenizer Initialization and Fitting:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Instructions'])

#Vocabulary Size and Maximum Sequence Length Calculation:
vocab_size = len(tokenizer.word_index) + 1
max_sequence_length = max(df['Instructions'].apply(lambda x: len(x.split())))

#Texts to Sequences:
sequences = tokenizer.texts_to_sequences(df['Instructions'])

#Padding Sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)


In [ ]:
# Assuming 'text' is your input data and 'label' is your target variable
X = data   #data
y = df['Title'].values           #labels 

# Convert labels to numerical format using LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [68]:
# Define the Feature Extractor
def build_feature_extractor(vocab_size,embedding_dim,max_sequence_length):
    model = models.Sequential([
        layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
        layers.Conv1D(filters=128, kernel_size=5, activation='relu'),
        layers.GlobalMaxPooling1D()
    ])
    return model

In [65]:
# Define the Label Predictor
def build_label_predictor(num_classes):
    model = models.Sequential([
        Dense(units=num_classes, activation='softmax',name='output')
    ])
    return model

In [69]:
# Define the Domain Predictor
def build_domain_predictor():
    model = models.Sequential([
        layers.Dense(64,activation='relu',name='dense_1'),
        layers.Dense(1,activation='sigmoid',name='dense_2')
    ])
    return model

In [70]:
# Build the complete DANN model
def build_dann(vocab_size,embedding_dim,max_sequence_length, num_classes):
    feature_extractor = build_feature_extractor(vocab_size,embedding_dim,max_sequence_length)
    label_predictor = build_label_predictor(num_classes)
    domain_predictor = build_domain_predictor()

    # Define inputs
    input_data = layers.Input(shape=(max_sequence_length,))
    label = layers.Input(shape=(num_classes,))
    domain_label = layers.Input(shape=(1,))
    
    # Feature extractor output
    feature_output = feature_extractor(input_data)
    
    # Label prediction branch
    label_output = label_predictor(feature_output)
    
    # Domain prediction branch
    domain_output = domain_predictor(feature_output)
    
    dann_model = models.Model(inputs=[input_data, label, domain_label], 
                              outputs=[label_output, domain_output])
    
    return dann_model

In [71]:
# Define loss functions
def label_loss(y_true, y_pred):
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred)

def domain_loss(y_true, y_pred):
    return tf.keras.losses.binary_crossentropy(y_true, y_pred)

In [60]:
# Build and compile the DANN model
input_shape = (28, 28, 1)
num_classes = 10

dann_model = build_dann(input_shape, num_classes)

dann_model.compile(optimizer='Adam',
                   loss=[label_loss, domain_loss],
                   loss_weights=[1.0,0.1])

ValueError: Exception encountered when calling layer "sequential_5" (type Sequential).

Input 0 of layer "global_max_pooling1d_5" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 28, 24, 128)

Call arguments received by layer "sequential_5" (type Sequential):
  • inputs=tf.Tensor(shape=(None, 28, 28, 1), dtype=float32)
  • training=None
  • mask=None